In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=0159eaf74bb76b5ccdeb70c4aeaec01399267f5755c7983a935ba5db79422d34
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


<font size=7><font color="orange">***创建SparkDataFrame***<font>

<font size=4>***3万字长文 PySpark入门级学习教程，框架思维: https://zhuanlan.zhihu.com/p/395431025***<font>

<font size=4>***1. 使用RDD来创建***<font>

In [5]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [12]:
conf = SparkConf().setAppName("test_SamShare").setMaster("local[4]")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/17 06:40:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
rdd = sc.parallelize([("Sam", 28, 88), ("Flora", 28, 90), ("Run", 1, 60)])
df = rdd.toDF(["name", "age", "score"])
df.show()
df.printSchema()

# +-----+---+-----+
# | name|age|score|
# +-----+---+-----+
# |  Sam| 28|   88|
# |Flora| 28|   90|
# |  Run|  1|   60|
# +-----+---+-----+
# root
#  |-- name: string (nullable = true)
#  |-- age: long (nullable = true)
#  |-- score: long (nullable = true)

PySparkRuntimeError: [CALL_BEFORE_INITIALIZE] Not supported to call `RDD.toDF` before initialize SparkSession.

In [18]:
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
rdd = spark.sparkContext.parallelize(data)
dfFromRDD1 = rdd.toDF()
dfFromRDD1.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)



<font size=4>***2.使用python的DataFrame来创建***<font>

In [20]:
df = pd.DataFrame([['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, 60]],
                  columns=['name', 'age', 'score'])
print(">> 打印DataFrame:")
print(df)
print("\n")
Spark_df = spark.createDataFrame(df)
print(">> 打印SparkDataFrame:")
Spark_df.show()
# >> 打印DataFrame:
#     name  age  score
# 0    Sam   28     88
# 1  Flora   28     90
# 2    Run    1     60
# >> 打印SparkDataFrame:
# +-----+---+-----+
# | name|age|score|
# +-----+---+-----+
# |  Sam| 28|   88|
# |Flora| 28|   90|
# |  Run|  1|   60|
# +-----+---+-----+

>> 打印DataFrame:
    name  age  score
0    Sam   28     88
1  Flora   28     90
2    Run    1     60


>> 打印SparkDataFrame:
+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1|   60|
+-----+---+-----+



<font size=4>***3.使用List来创建***<font>

In [21]:
list_values = [['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, 60]]
Spark_df = spark.createDataFrame(list_values, ['name', 'age', 'score'])
Spark_df.show()
# +-----+---+-----+
# | name|age|score|
# +-----+---+-----+
# |  Sam| 28|   88|
# |Flora| 28|   90|
# |  Run|  1|   60|
# +-----+---+-----+

+-----+---+-----+
| name|age|score|
+-----+---+-----+
|  Sam| 28|   88|
|Flora| 28|   90|
|  Run|  1|   60|
+-----+---+-----+



<font size=4>***4. 读取数据文件来创建***<font>

In [ ]:
# 4.1 CSV文件
df = spark.read.option("header", "true")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv("./test/data/titanic/train.csv")
df.show(5)
df.printSchema()

# 4.2 json文件
df = spark.read.json("./test/data/hello_samshare.json")
df.show(5)
df.printSchema()

<font size=4>***5. 通过读取数据库来创建***<font>

In [23]:
# 5.1 读取hive数据
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH 'data/kv1.txt' INTO TABLE src")
df = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")
df.show(5)

# 5.2 读取mysql数据
url = "jdbc:mysql://localhost:3306/test"
df = spark.read.format("jdbc") \
 .option("url", url) \
 .option("dbtable", "runoob_tbl") \
 .option("user", "root") \
 .option("password", "8888") \
 .load()
df.show()

AnalysisException: [NOT_SUPPORTED_COMMAND_WITHOUT_HIVE_SUPPORT] CREATE Hive TABLE (AS SELECT) is not supported, if you want to enable it, please set "spark.sql.catalogImplementation" to "hive".;
'CreateTable `spark_catalog`.`default`.`src`, Ignore


<font size=7><font color="orange">***常用的SparkDataFrame API***<font>

In [26]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
# SparkSQL的许多功能封装在SparkSession的方法接口中, SparkContext则不行的。
spark = SparkSession.builder \
    .appName("sam_SamShare") \
    .config("master", "local[4]") \
    .enableHiveSupport() \
    .getOrCreate()
sc = spark.sparkContext

# 创建一个SparkDataFrame
rdd = sc.parallelize([("Sam", 28, 88, "M"),
                      ("Flora", 28, 90, "F"),
                      ("Run", 1, 60, None),
                      ("Peter", 55, 100, "M"),
                      ("Mei", 54, 95, "F")])
df = rdd.toDF(["name", "age", "score", "sex"])
df.show()
df.printSchema()

# +-----+---+-----+----+
# | name|age|score| sex|
# +-----+---+-----+----+
# |  Sam| 28|   88|   M|
# |Flora| 28|   90|   F|
# |  Run|  1|   60|null|
# |Peter| 55|  100|   M|
# |  Mei| 54|   95|   F|
# +-----+---+-----+----+
# root
#  |-- name: string (nullable = true)
#  |-- age: long (nullable = true)
#  |-- score: long (nullable = true)
#  |-- sex: string (nullable = true)

23/10/17 06:48:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|NULL|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- score: long (nullable = true)
 |-- sex: string (nullable = true)



<font size=4>***1. 查看DataFrame的APIs***<font>

In [27]:
# DataFrame.collect
# 以列表形式返回行
df.collect()
# [Row(name='Sam', age=28, score=88, sex='M'),
# Row(name='Flora', age=28, score=90, sex='F'),
# Row(name='Run', age=1, score=60, sex=None),
# Row(name='Peter', age=55, score=100, sex='M'),
# Row(name='Mei', age=54, score=95, sex='F')]

[Row(name='Sam', age=28, score=88, sex='M'),
 Row(name='Flora', age=28, score=90, sex='F'),
 Row(name='Run', age=1, score=60, sex=None),
 Row(name='Peter', age=55, score=100, sex='M'),
 Row(name='Mei', age=54, score=95, sex='F')]

In [28]:
# DataFrame.count
df.count()
# 5

5

In [29]:
# DataFrame.columns
df.columns
# ['name', 'age', 'score', 'sex']

['name', 'age', 'score', 'sex']

In [30]:
# DataFrame.dtypes
df.dtypes
# [('name', 'string'), ('age', 'bigint'), ('score', 'bigint'), ('sex', 'string')]

[('name', 'string'), ('age', 'bigint'), ('score', 'bigint'), ('sex', 'string')]

In [31]:
# DataFrame.describe
# 返回列的基础统计信息
df.describe(['age']).show()
# +-------+------------------+
# |summary|               age|
# +-------+------------------+
# |  count|                 5|
# |   mean|              33.2|
# | stddev|22.353970564532826|
# |    min|                 1|
# |    max|                55|
# +-------+------------------+
df.describe().show()
# +-------+-----+------------------+------------------+----+
# |summary| name|               age|             score| sex|
# +-------+-----+------------------+------------------+----+
# |  count|    5|                 5|                 5|   4|
# |   mean| null|              33.2|              86.6|null|
# | stddev| null|22.353970564532826|15.582040944625966|null|
# |    min|Flora|                 1|                60|   F|
# |    max|  Sam|                55|               100|   M|
# +-------+-----+------------------+------------------+----+

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 5|
|   mean|              33.2|
| stddev|22.353970564532826|
|    min|                 1|
|    max|                55|
+-------+------------------+



23/10/17 06:50:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----+------------------+------------------+----+
|summary| name|               age|             score| sex|
+-------+-----+------------------+------------------+----+
|  count|    5|                 5|                 5|   4|
|   mean| NULL|              33.2|              86.6|NULL|
| stddev| NULL|22.353970564532826|15.582040944625966|NULL|
|    min|Flora|                 1|                60|   F|
|    max|  Sam|                55|               100|   M|
+-------+-----+------------------+------------------+----+



In [32]:
# DataFrame.select
# 选定指定列并按照一定顺序呈现
df.select("sex", "score").show()

+----+-----+
| sex|score|
+----+-----+
|   M|   88|
|   F|   90|
|NULL|   60|
|   M|  100|
|   F|   95|
+----+-----+



In [33]:
# DataFrame.first
# DataFrame.head
# 查看第1条数据
df.first()
# Row(name='Sam', age=28, score=88, sex='M')
df.head(1)
# [Row(name='Sam', age=28, score=88, sex='M')]

[Row(name='Sam', age=28, score=88, sex='M')]

In [34]:
# DataFrame.freqItems
# 查看指定列的枚举值
df.freqItems(["age","sex"]).show()
# +---------------+-------------+
# |  age_freqItems|sex_freqItems|
# +---------------+-------------+
# |[55, 1, 28, 54]|      [M, F,]|
# +---------------+-------------+


+---------------+-------------+
|  age_freqItems|sex_freqItems|
+---------------+-------------+
|[55, 1, 28, 54]| [M, NULL, F]|
+---------------+-------------+



In [35]:
# DataFrame.summary
df.summary().show()
# +-------+-----+------------------+------------------+----+
# |summary| name|               age|             score| sex|
# +-------+-----+------------------+------------------+----+
# |  count|    5|                 5|                 5|   4|
# |   mean| null|              33.2|              86.6|null|
# | stddev| null|22.353970564532826|15.582040944625966|null|
# |    min|Flora|                 1|                60|   F|
# |    25%| null|                28|                88|null|
# |    50%| null|                28|                90|null|
# |    75%| null|                54|                95|null|
# |    max|  Sam|                55|               100|   M|
# +-------+-----+------------------+------------------+----+

+-------+-----+------------------+------------------+----+
|summary| name|               age|             score| sex|
+-------+-----+------------------+------------------+----+
|  count|    5|                 5|                 5|   4|
|   mean| NULL|              33.2|              86.6|NULL|
| stddev| NULL|22.353970564532826|15.582040944625966|NULL|
|    min|Flora|                 1|                60|   F|
|    25%| NULL|                28|                88|NULL|
|    50%| NULL|                28|                90|NULL|
|    75%| NULL|                54|                95|NULL|
|    max|  Sam|                55|               100|   M|
+-------+-----+------------------+------------------+----+



In [36]:
# DataFrame.sample
# 按照一定规则从df随机抽样数据
df.sample(0.5).show()
# +-----+---+-----+----+
# | name|age|score| sex|
# +-----+---+-----+----+
# |  Sam| 28|   88|   M|
# |  Run|  1|   60|null|
# |Peter| 55|  100|   M|
# +-----+---+-----+----+

+----+---+-----+----+
|name|age|score| sex|
+----+---+-----+----+
| Run|  1|   60|NULL|
| Mei| 54|   95|   F|
+----+---+-----+----+



<font size=4>***2. 简单处理DataFrame的APIs***<font>

In [37]:
# DataFrame.distinct
# 对数据集进行去重
df.distinct().show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Mei| 54|   95|   F|
|Peter| 55|  100|   M|
|  Run|  1|   60|NULL|
+-----+---+-----+----+



In [38]:
# DataFrame.dropDuplicates
# 对指定列去重
df.dropDuplicates(["sex"]).show()
# +-----+---+-----+----+
# | name|age|score| sex|
# +-----+---+-----+----+
# |Flora| 28|   90|   F|
# |  Run|  1|   60|null|
# |  Sam| 28|   88|   M|
# +-----+---+-----+----+

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Run|  1|   60|NULL|
|Flora| 28|   90|   F|
|  Sam| 28|   88|   M|
+-----+---+-----+----+



In [39]:
# DataFrame.exceptAll
# DataFrame.subtract
# 根据指定的df对df进行去重
df1 = spark.createDataFrame(
        [("a", 1), ("a", 1), ("b",  3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["C1", "C2"])
df3 = df1.exceptAll(df2)  # 没有去重的功效
df4 = df1.subtract(df2)  # 有去重的奇效
df1.show()
df2.show()
df3.show()
df4.show()
# +---+---+
# | C1| C2|
# +---+---+
# |  a|  1|
# |  a|  1|
# |  b|  3|
# |  c|  4|
# +---+---+
# +---+---+
# | C1| C2|
# +---+---+
# |  a|  1|
# |  b|  3|
# +---+---+
# +---+---+
# | C1| C2|
# +---+---+
# |  a|  1|
# |  c|  4|
# +---+---+
# +---+---+
# | C1| C2|
# +---+---+
# |  c|  4|
# +---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  a|  1|
|  b|  3|
|  c|  4|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  b|  3|
+---+---+



+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  c|  4|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  c|  4|
+---+---+



In [41]:
# DataFrame.intersectAll
# 返回两个DataFrame的交集
df1 = spark.createDataFrame(
        [("a", 1), ("a", 1), ("b",  3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("a", 1), ("b", 4)], ["C1", "C2"])
df1.intersectAll(df2).show()
# +---+---+
# | C1| C2|
# +---+---+
# |  a|  1|
# +---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  1|
+---+---+



In [44]:
# DataFrame.drop
# 丢弃指定列
df.drop('age').show()

+-----+-----+----+
| name|score| sex|
+-----+-----+----+
|  Sam|   88|   M|
|Flora|   90|   F|
|  Run|   60|NULL|
|Peter|  100|   M|
|  Mei|   95|   F|
+-----+-----+----+



In [45]:
# DataFrame.withColumn
# 新增列
df1 = df.withColumn("birth_year", 2021 - df.age)
df1.show()
# +-----+---+-----+----+----------+
# | name|age|score| sex|birth_year|
# +-----+---+-----+----+----------+
# |  Sam| 28|   88|   M|      1993|
# |Flora| 28|   90|   F|      1993|
# |  Run|  1|   60|null|      2020|
# |Peter| 55|  100|   M|      1966|
# |  Mei| 54|   95|   F|      1967|
# +-----+---+-----+----+----------+

+-----+---+-----+----+----------+
| name|age|score| sex|birth_year|
+-----+---+-----+----+----------+
|  Sam| 28|   88|   M|      1993|
|Flora| 28|   90|   F|      1993|
|  Run|  1|   60|NULL|      2020|
|Peter| 55|  100|   M|      1966|
|  Mei| 54|   95|   F|      1967|
+-----+---+-----+----+----------+



In [46]:
# DataFrame.withColumnRenamed
# 重命名列名
df1 = df.withColumnRenamed("sex", "gender")
df1.show()
# +-----+---+-----+------+
# | name|age|score|gender|
# +-----+---+-----+------+
# |  Sam| 28|   88|     M|
# |Flora| 28|   90|     F|
# |  Run|  1|   60|  null|
# |Peter| 55|  100|     M|
# |  Mei| 54|   95|     F|
# +-----+---+-----+------+

+-----+---+-----+------+
| name|age|score|gender|
+-----+---+-----+------+
|  Sam| 28|   88|     M|
|Flora| 28|   90|     F|
|  Run|  1|   60|  NULL|
|Peter| 55|  100|     M|
|  Mei| 54|   95|     F|
+-----+---+-----+------+



In [47]:
# DataFrame.dropna
# 丢弃空值，DataFrame.dropna(how='any', thresh=None, subset=None)
df.dropna(how='all', subset=['sex']).show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |  Sam| 28|   88|  M|
# |Flora| 28|   90|  F|
# |Peter| 55|  100|  M|
# |  Mei| 54|   95|  F|
# +-----+---+-----+---+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



In [48]:
# DataFrame.fillna
# 空值填充操作
df1 = spark.createDataFrame(
        [("a", None), ("a", 1), (None,  3), ("c", 4)], ["C1", "C2"])
# df2 = df1.na.fill({"C1": "d", "C2": 99})
df2 = df1.fillna({"C1": "d", "C2": 99})
df1.show()
df2.show()

+----+----+
|  C1|  C2|
+----+----+
|   a|NULL|
|   a|   1|
|NULL|   3|
|   c|   4|
+----+----+

+---+---+
| C1| C2|
+---+---+
|  a| 99|
|  a|  1|
|  d|  3|
|  c|  4|
+---+---+



In [49]:
# DataFrame.filter
# 根据条件过滤
df.filter(df.age>50).show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |Peter| 55|  100|  M|
# |  Mei| 54|   95|  F|
# +-----+---+-----+---+
df.where(df.age==28).show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |  Sam| 28|   88|  M|
# |Flora| 28|   90|  F|
# +-----+---+-----+---+
df.filter("age<18").show()
# +----+---+-----+----+
# |name|age|score| sex|
# +----+---+-----+----+
# | Run|  1|   60|null|
# +----+---+-----+----+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
+-----+---+-----+---+

+----+---+-----+----+
|name|age|score| sex|
+----+---+-----+----+
| Run|  1|   60|NULL|
+----+---+-----+----+



In [50]:
# DataFrame.join
# 这个不用多解释了，直接上案例来看看具体的语法即可，DataFrame.join(other, on=None, how=None)
df1 = spark.createDataFrame(
        [("a", 1), ("d", 1), ("b",  3), ("c", 4)], ["id", "num1"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["id", "num2"])
df1.join(df2, df1.id == df2.id, 'left').select(df1.id.alias("df1_id"),
                                               df1.num1.alias("df1_num"),
                                               df2.num2.alias("df2_num")
                                               ).sort(["df1_id"], ascending=False)\
    .show()

+------+-------+-------+
|df1_id|df1_num|df2_num|
+------+-------+-------+
|     d|      1|   NULL|
|     c|      4|   NULL|
|     b|      3|      3|
|     a|      1|      1|
+------+-------+-------+



In [51]:
# DataFrame.agg(*exprs)
# 聚合数据，可以写多个聚合方法，如果不写groupBy的话就是对整个DF进行聚合
# DataFrame.alias
# 设置列或者DataFrame别名
# DataFrame.groupBy
# 根据某几列进行聚合，如有多列用列表写在一起，如 df.groupBy(["sex", "age"])
df.groupBy("sex").agg(F.min(df.age).alias("最小年龄"),
                      F.expr("avg(age)").alias("平均年龄"),
                      F.expr("collect_list(name)").alias("姓名集合")
                      ).show()
# +----+--------+--------+------------+
# | sex|最小年龄|平均年龄|    姓名集合|
# +----+--------+--------+------------+
# |   F|      28|    41.0|[Flora, Mei]|
# |null|       1|     1.0|       [Run]|
# |   M|      28|    41.5|[Sam, Peter]|
# +----+--------+--------+------------+

+----+--------+--------+------------+
| sex|最小年龄|平均年龄|    姓名集合|
+----+--------+--------+------------+
|   M|      28|    41.5|[Sam, Peter]|
|   F|      28|    41.0|[Flora, Mei]|
|NULL|       1|     1.0|       [Run]|
+----+--------+--------+------------+



In [52]:
# DataFrame.foreach
# 对每一行进行函数方法的应用
def f(person):
    print(person.name)
df.foreach(f)
# Peter
# Run
# Sam
# Flora
# Mei

Sam
Peter
Mei
Run
Flora


In [53]:
# DataFrame.replace
# 修改df里的某些值
df1 = df.na.replace({"M": "Male", "F": "Female"})
df1.show()

+-----+---+-----+------+
| name|age|score|   sex|
+-----+---+-----+------+
|  Sam| 28|   88|  Male|
|Flora| 28|   90|Female|
|  Run|  1|   60|  NULL|
|Peter| 55|  100|  Male|
|  Mei| 54|   95|Female|
+-----+---+-----+------+



In [54]:
# DataFrame.union
# 相当于SQL里的union all操作
df1 = spark.createDataFrame(
        [("a", 1), ("d", 1), ("b",  3), ("c", 4)], ["id", "num"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["id", "num"])
df1.union(df2).show()
df1.unionAll(df2).show()
# 这里union没有去重，不知道为啥，有知道的朋友麻烦解释下，谢谢了。
# +---+---+
# | id|num|
# +---+---+
# |  a|  1|
# |  d|  1|
# |  b|  3|
# |  c|  4|
# |  a|  1|
# |  b|  3|
# +---+---+

+---+---+
| id|num|
+---+---+
|  a|  1|
|  d|  1|
|  b|  3|
|  c|  4|
|  a|  1|
|  b|  3|
+---+---+

+---+---+
| id|num|
+---+---+
|  a|  1|
|  d|  1|
|  b|  3|
|  c|  4|
|  a|  1|
|  b|  3|
+---+---+



In [55]:
# DataFrame.unionByName
# 根据列名来进行合并数据集
df1 = spark.createDataFrame([[1, 2, 3]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col0"])
df1.unionByName(df2).show()
# +----+----+----+
# |col0|col1|col2|
# +----+----+----+
# |   1|   2|   3|
# |   6|   4|   5|
# +----+----+----+

+----+----+----+
|col0|col1|col2|
+----+----+----+
|   1|   2|   3|
|   6|   4|   5|
+----+----+----+



<font size=4>***3. DataFrame的列操作APIs***<font>

这里主要针对的是列进行操作，比如说重命名、排序、空值判断、类型判断等，这里就不展开写demo了，看看语法应该大家都懂了。

In [ ]:
Column.alias(*alias, **kwargs)  # 重命名列名
Column.asc()  # 按照列进行升序排序
Column.desc()  # 按照列进行降序排序
Column.astype(dataType)  # 类型转换
Column.cast(dataType)  # 强制转换类型
Column.between(lowerBound, upperBound)  # 返回布尔值，是否在指定区间范围内
Column.contains(other)  # 是否包含某个关键词
Column.endswith(other)  # 以什么结束的值，如 df.filter(df.name.endswith('ice')).collect()
Column.isNotNull()  # 筛选非空的行
Column.isNull()
Column.isin(*cols)  # 返回包含某些值的行 df[df.name.isin("Bob", "Mike")].collect()
Column.like(other)  # 返回含有关键词的行
Column.when(condition, value)  # 给True的赋值
Column.otherwise(value)  # 与when搭配使用，df.select(df.name, F.when(df.age > 3, 1).otherwise(0)).show()
Column.rlike(other)  # 可以使用正则的匹配 df.filter(df.name.rlike('ice$')).collect()
Column.startswith(other)  # df.filter(df.name.startswith('Al')).collect()
Column.substr(startPos, length)  # df.select(df.name.substr(1, 3).alias("col")).collect()

<font size=4>***4. DataFrame的一些思路变换操作APIs***<font>

In [56]:
# DataFrame.createOrReplaceGlobalTempView
# DataFrame.dropGlobalTempView
# 创建全局的试图，注册后可以使用sql语句来进行操作，生命周期取决于Spark application本身
df.createOrReplaceGlobalTempView("people")
spark.sql("select * from global_temp.people where sex = 'M' ").show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |  Sam| 28|   88|  M|
# |Peter| 55|  100|  M|
# +-----+---+-----+---+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Peter| 55|  100|  M|
+-----+---+-----+---+



In [57]:
# DataFrame.createOrReplaceTempView
# DataFrame.dropTempView
# 创建本地临时试图，生命周期取决于用来创建此数据集的SparkSession
df.createOrReplaceTempView("tmp_people")
spark.sql("select * from tmp_people where sex = 'F' ").show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |Flora| 28|   90|  F|
# |  Mei| 54|   95|  F|
# +-----+---+-----+---+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|Flora| 28|   90|  F|
|  Mei| 54|   95|  F|
+-----+---+-----+---+



In [59]:
# DataFrame.cache\DataFrame.persist
# 可以把一些数据放入缓存中，default storage level (MEMORY_AND_DISK).
df.cache()
df.persist()
df.unpersist()

23/10/17 06:58:43 WARN CacheManager: Asked to cache already cached data.


DataFrame[name: string, age: bigint, score: bigint, sex: string]

In [61]:
# DataFrame.crossJoin
# 返回两个DataFrame的笛卡尔积关联的DataFrame
df1 = df.select("name", "sex")
df2 = df.select("name", "sex")
df3 = df1.crossJoin(df2)
print("表1的记录数", df1.count())
print("表2的记录数", df2.count())
print("笛卡尔积后的记录数", df3.count())
# 表1的记录数 5
# 表2的记录数 5
# 笛卡尔积后的记录数 25

表1的记录数 5
表2的记录数 5


笛卡尔积后的记录数 25


In [62]:
# DataFrame.toPandas
# 把SparkDataFrame转为 Pandas的DataFrame
df.toPandas()

,name,age,score,sex
0,Sam,28,88,M
1,Flora,28,90,F
2,Run,1,60,None
3,Peter,55,100,M
4,Mei,54,95,F


In [63]:
# DataFrame.rdd
# 把SparkDataFrame转为rdd，这样子可以用rdd的语法来操作数据
df.rdd

MapPartitionsRDD[230] at javaToPython at NativeMethodAccessorImpl.java:0

<font size=4>***5. DataFrame的一些统计操作APIs***<font>

In [65]:
df.show()

+-----+---+-----+----+
| name|age|score| sex|
+-----+---+-----+----+
|  Sam| 28|   88|   M|
|Flora| 28|   90|   F|
|  Run|  1|   60|NULL|
|Peter| 55|  100|   M|
|  Mei| 54|   95|   F|
+-----+---+-----+----+



In [64]:
# DataFrame.cov
# 计算指定两列的样本协方差
df.cov("age", "score")
# 324.59999999999997

324.6000000000001

In [66]:
# DataFrame.corr
# 计算指定两列的相关系数，DataFrame.corr(col1, col2, method=None)，目前method只支持Pearson相关系数
df.corr("age", "score", method="pearson")
# 0.9319004030498815

0.9319004030498816

In [67]:
# DataFrame.cube
# 创建多维度聚合的结果，通常用于分析数据，比如我们指定两个列进行聚合，比如name和age，那么这个函数返回的聚合结果会
# groupby("name", "age")
# groupby("name")
# groupby("age")
# groupby(all)
# 四个聚合结果的union all 的结果

In [68]:
df1 = df.filter(df.name != "Run")
print(df1.show())
df1.cube("name", "sex").count().show()
# +-----+---+-----+---+
# | name|age|score|sex|
# +-----+---+-----+---+
# |  Sam| 28|   88|  M|
# |Flora| 28|   90|  F|
# |Peter| 55|  100|  M|
# |  Mei| 54|   95|  F|
# +-----+---+-----+---+
# cube 聚合之后的结果
# +-----+----+-----+
# | name| sex|count|
# +-----+----+-----+
# | null|   F|    2|
# | null|null|    4|
# |Flora|null|    1|
# |Peter|null|    1|
# | null|   M|    2|
# |Peter|   M|    1|
# |  Sam|   M|    1|
# |  Sam|null|    1|
# |  Mei|   F|    1|
# |  Mei|null|    1|
# |Flora|   F|    1|
# +-----+----+-----+

+-----+---+-----+---+
| name|age|score|sex|
+-----+---+-----+---+
|  Sam| 28|   88|  M|
|Flora| 28|   90|  F|
|Peter| 55|  100|  M|
|  Mei| 54|   95|  F|
+-----+---+-----+---+

None
+-----+----+-----+
| name| sex|count|
+-----+----+-----+
|  Sam|   M|    1|
|  Sam|NULL|    1|
| NULL|NULL|    4|
| NULL|   M|    2|
| NULL|   F|    2|
|Flora|   F|    1|
|Flora|NULL|    1|
|  Mei|   F|    1|
|Peter|   M|    1|
|Peter|NULL|    1|
|  Mei|NULL|    1|
+-----+----+-----+



<font size=7><font color="orange">***保存数据/写入数据库***<font>

这里的保存数据主要是保存到Hive中的栗子，主要包括了overwrite、append等方式。

<font size=4>***1. 当结果集为SparkDataFrame的时候***<font>

In [6]:
spark = SparkSession.builder \
    .appName("sam_SamShare") \
    .config("master", "local[4]") \
    .enableHiveSupport() \
    .getOrCreate()
# sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/17 09:01:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
import pandas as pd
from datetime import datetime
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import HiveContext

conf = SparkConf()\
      .setAppName("test")\
      .set("hive.exec.dynamic.partition.mode", "nonstrict") # 动态写入hive分区表
sc = SparkContext(conf=conf)
hc = HiveContext(sc)
sc.setLogLevel("ERROR")
    
list_values = [['Sam', 28, 88], ['Flora', 28, 90], ['Run', 1, 60]]
Spark_df = spark.createDataFrame(list_values, ['name', 'age', 'score'])
print(Spark_df.show())
save_table = "tmp.samshare_pyspark_savedata"

# 方式1:直接写入到Hive
Spark_df.write.format("hive").mode("overwrite").saveAsTable(save_table) # 或者改成append模式
print(datetime.now().strftime("%y/%m/%d %H:%M:%S"), "测试数据写入到表" + save_table)

# 方式2:注册为临时表，使用SparkSQL来写入分区表
Spark_df.createOrReplaceTempView("tmp_table")
write_sql = """
insert overwrite table {0} partitions (pt_date='{1}')
select * from tmp_table
""".format(save_table, "20210520")
hc.sql(write_sql)
print(datetime.now().strftime("%y/%m/%d %H:%M:%S"), "测试数据写入到表" + save_table)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=sam_SamShare, master=local[*]) created by getOrCreate at /tmp/ipykernel_33/2776350081.py:5 

<font size=4>***2. 当结果集为Python的DataFrame的时候***<font>

In [70]:
import pandas as pd
from datetime import datetime
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import HiveContext

conf = SparkConf()\
      .setAppName("test")\
      .set("hive.exec.dynamic.partition.mode", "nonstrict") # 动态写入hive分区表
sc = SparkContext(conf=conf)
hc = HiveContext(sc)
sc.setLogLevel("ERROR")
    
result_df = pd.DataFrame([1,2,3], columns=['a'])
save_table = "tmp.samshare_pyspark_savedata"

# 获取DataFrame的schema
c1 = list(result_df.columns)
# 转为SparkDataFrame
result = hc.createDataFrame(result_df.astype(str), c1)
result.write.format("hive").mode("overwrite").saveAsTable(save_table) # 或者改成append模式
print(datetime.now().strftime("%y/%m/%d %H:%M:%S"), "测试数据写入到表" + save_table)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test_SamShare, master=local[4]) created by __init__ at /tmp/ipykernel_32/1162219135.py:2 